In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import numpy as np

# Carga de datos

In [2]:
train_data = pd.read_csv("../spaceship-titanic/train.csv")
print(train_data.shape)
train_data.head(3)

(8693, 14)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


In [3]:
test_data = pd.read_csv("../spaceship-titanic/test.csv")
test_data.shape

(4277, 13)

In [4]:
train_data['Dataset'] = 'train'
test_data['Dataset'] = 'test'
data = pd.concat([train_data, test_data], ignore_index=True)
data.shape

(12970, 15)

In [5]:
data.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported      object
Dataset          object
dtype: object

# ETL Process

## Tamaño del grupo del pasajero

In [6]:
data[['PassengerGroup','PassengerGroupNum']] = data.PassengerId.str.split('_', expand=True)

In [7]:
group_size = data.groupby('PassengerGroup')[['PassengerGroupNum']].max().rename(columns={'PassengerGroupNum':'GroupSize'}).reset_index()
group_size['GroupSize'] = group_size.GroupSize.astype(int)
data = data.merge(group_size, how='left', on='PassengerGroup')

## Nueva categoría para los nulos en `HomePlanet`

Imputamos algunos nulos con base en el exploratorio realizado.

In [8]:
homePlanetByPassGroup = data[data.HomePlanet.notna()].groupby('PassengerGroup')[['HomePlanet']].first()
data = data.drop(columns=['HomePlanet']).merge(homePlanetByPassGroup, on='PassengerGroup', how='left')

In [9]:
data.HomePlanet.fillna("Ns/Nc", inplace=True)

## Cabin deck and cabin side

In [10]:
data[['CabinDeck','CabinSide']] = data.Cabin.str.split('/', expand=True)[[0,2]]
data[['CabinDeck','CabinSide']] = data[['CabinDeck','CabinSide']].astype(str)

## Nueva categoría para los nulos en `CryoSleep`

Imputamos algunos nulos con base en el exploratorio realizado.

In [11]:
target_idxs = data[ (data.CryoSleep.isna()) & (data.CabinDeck == 'T') ].index
data.loc[target_idxs, 'CryoSleep'] = False

In [12]:
data['CryoSleep'] = data.CryoSleep.astype('string').fillna('Ns/Nc')

## Nueva categoría para los nulos en `Destination`

In [13]:
data.Destination.fillna("Ns/Nc", inplace=True)

## Nueva categoría para los nulos en `VIP`

Imputamos algunos nulos con base en el exploratorio realizado.

In [14]:
target_idxs = data[ (data.VIP.isna()) & (data.CabinDeck.isin(['G','T'])) ].index
data.loc[target_idxs, 'VIP'] = False

In [15]:
data['VIP'] = data.VIP.astype('string').fillna('Ns/Nc')

## Nivel de gasto total

In [16]:
#data['RoomService'].fillna(.0, inplace=True)
#data['FoodCourt'].fillna(.0, inplace=True)
#data['ShoppingMall'].fillna(.0, inplace=True)
#data['Spa'].fillna(.0, inplace=True)
#data['VRDeck'].fillna(.0, inplace=True)

data['AnyExpenses'] = (data[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1) != 0).astype(str)
data['EssencialBill'] = data[['FoodCourt','ShoppingMall']].sum(axis=1)
data['NonEssencialBill'] = data[['RoomService','Spa','VRDeck']].sum(axis=1)

#quantiles = data.loc[data.EssencialBill!=0, 'EssencialBill'].quantile([0.33, 0.66, 0.95]).values
#bins = np.append( np.insert(quantiles, 0, [0, 0.5]), np.inf )
#data['EssencialBill'] = pd.cut(data.EssencialBill, bins, include_lowest=True, labels=[0,1,2,3,4])

#quantiles = data.loc[data.NonEssencialBill!=0, 'NonEssencialBill'].quantile([0.33, 0.66, 0.95]).values
#bins = np.append( np.insert(quantiles, 0, [0, 0.5]), np.inf )
#data['NonEssencialBill'] = pd.cut(data.NonEssencialBill, bins, include_lowest=True, labels=[0,1,2,3,4])

## Rango de edad

En primer lugar, calculamos la mediana de edad en función de la variable `TotalExpensesBinarized` para los pasajeros que no tienen la edad informada.

In [17]:
data['TotalBill'] = data[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)

quantiles = data.loc[data.TotalBill!=0, 'TotalBill'].quantile([0.33, 0.66, 0.95]).values
bins = np.append( np.insert(quantiles, 0, [0, 0.5]), np.inf )
data['TotalBill'] = pd.cut(data.TotalBill, bins, include_lowest=True, labels=[0,1,2,3,4])

ageInfoByTotalBillLabel = data.groupby('TotalBill').agg({'Age':['mean','std']})
ageInfoByTotalBillLabel.columns = [f"{x}{y.capitalize()}" for x, y in ageInfoByTotalBillLabel.columns.to_flat_index()]
ageInfoByTotalBillLabel.reset_index(inplace=True)
ageInfoByTotalBillLabel

,TotalBill,AgeMean,AgeStd
0,0,25.105480,15.823308
1,1,28.961475,12.035931
2,2,31.147156,12.669119
3,3,33.634283,12.626012
4,4,36.717808,11.962158


In [18]:
print(data.Age.isna().sum())

for _, (totalBillLbl, ageMean, ageStd) in ageInfoByTotalBillLabel.iterrows():
    target_idxs = data[ (data.Age.isna()) & (data.TotalBill==totalBillLbl) ].index
    
    normal_distribution = pd.Series( np.random.normal(loc=ageMean, scale=ageStd, size=len(target_idxs)), index=target_idxs )
    #print(normal_distribution)
    data.Age.fillna(normal_distribution, inplace=True)
    
    print(len(target_idxs), data.Age.isna().sum())

270
129 141
44 97
49 48
37 11
11 0


In [19]:
data['Age'] = pd.cut(data.Age, bins=[-np.inf, 15, 30, 45, 60, 75, np.inf], labels=[0, 1, 2, 3, 4, 5])

## Descartamos las variables sobrantes

In [20]:
data.dtypes

PassengerId            object
CryoSleep              string
Cabin                  object
Destination            object
Age                  category
VIP                    string
RoomService           float64
FoodCourt             float64
ShoppingMall          float64
Spa                   float64
VRDeck                float64
Name                   object
Transported            object
Dataset                object
PassengerGroup         object
PassengerGroupNum      object
GroupSize               int64
HomePlanet             object
CabinDeck              object
CabinSide              object
AnyExpenses            object
EssencialBill         float64
NonEssencialBill      float64
TotalBill            category
dtype: object

In [21]:
data.drop(columns=['PassengerId','Cabin','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Name',
                   'PassengerGroup','PassengerGroupNum','TotalBill'], inplace=True)

data['Age'] = data.Age.astype(int)
data['AnyExpenses'] = data.AnyExpenses.astype(str)

# Conjuntos de train y test

In [22]:
data_train = data[ data.Dataset=='train' ].drop(columns=['Dataset'])
data_test = data[ data.Dataset=='test' ].drop(columns=['Dataset'])
data_train.shape, data_test.shape

((8693, 12), (4277, 12))

In [23]:
data_train.to_csv("data/processed_train.csv", index=False)
data_test.to_csv("data/processed_test.csv", index=False)

In [24]:
data.dtypes

CryoSleep            string
Destination          object
Age                   int64
VIP                  string
Transported          object
Dataset              object
GroupSize             int64
HomePlanet           object
CabinDeck            object
CabinSide            object
AnyExpenses          object
EssencialBill       float64
NonEssencialBill    float64
dtype: object

In [25]:
data.sample(3)

,CryoSleep,Destination,Age,VIP,Transported,Dataset,GroupSize,HomePlanet,CabinDeck,CabinSide,AnyExpenses,EssencialBill,NonEssencialBill
2181,True,55 Cancri e,2,False,True,train,2,Europa,A,S,False,0.0,0.0
6025,False,TRAPPIST-1e,2,False,True,train,4,Europa,B,P,True,3772.0,783.0
10223,True,TRAPPIST-1e,3,False,NaN,test,2,Mars,F,S,False,0.0,0.0
